In [63]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import numpy.matlib
from sklearn.model_selection import StratifiedKFold
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.layers import Flatten,Input,concatenate

In [64]:
data_train = pd.read_csv('train_fNxu4vz.csv')
data_sub = pd.read_csv('test_fjtUOL8.csv')
#data_sub.head()

In [65]:
min_max_scalerL = preprocessing.MinMaxScaler()
data_train['Loan_Amount_Requested'] = min_max_scalerL.fit_transform( data_train['Loan_Amount_Requested'].str.replace(',','').astype(int).values.reshape(-1, 1) )

data_sub['Loan_Amount_Requested'] = min_max_scalerL.transform( data_sub['Loan_Amount_Requested'].str.replace(',','').astype(int).values.reshape(-1, 1) )

In [66]:
min_max_scalerA = preprocessing.MinMaxScaler()
data_train['Annual_Income'] = min_max_scalerA.fit_transform( data_train['Annual_Income'].fillna(-1).values.reshape(-1, 1) )

data_sub['Annual_Income'] = min_max_scalerA.transform( data_sub['Annual_Income'].fillna(-1).values.reshape(-1, 1) )

min_max_scalerD = preprocessing.MinMaxScaler()
data_train['Debt_To_Income'] = min_max_scalerD.fit_transform( data_train['Debt_To_Income'].fillna(-1).values.reshape(-1, 1) )

data_sub['Debt_To_Income'] = min_max_scalerD.transform( data_sub['Debt_To_Income'].fillna(-1).values.reshape(-1, 1) )

min_max_scalerI = preprocessing.MinMaxScaler()
data_train['Inquiries_Last_6Mo'] = min_max_scalerI.fit_transform(  data_train['Inquiries_Last_6Mo'].fillna(-1).values.reshape(-1, 1) )

data_sub['Inquiries_Last_6Mo'] = min_max_scalerI.transform(  data_sub['Inquiries_Last_6Mo'].fillna(-1).values.reshape(-1, 1) )

min_max_scalerM = preprocessing.MinMaxScaler()
data_train['Months_Since_Deliquency'] = min_max_scalerM.fit_transform( data_train['Months_Since_Deliquency'].fillna(-1).values.reshape(-1, 1))

data_sub['Months_Since_Deliquency'] = min_max_scalerM.transform( data_sub['Months_Since_Deliquency'].fillna(-1).values.reshape(-1, 1))

min_max_scalerN = preprocessing.MinMaxScaler()
data_train['Number_Open_Accounts'] = min_max_scalerN.fit_transform( data_train['Number_Open_Accounts'].fillna(-1).values.reshape(-1, 1) )

data_sub['Number_Open_Accounts'] = min_max_scalerN.transform( data_sub['Number_Open_Accounts'].fillna(-1).values.reshape(-1, 1) )

min_max_scalerT = preprocessing.MinMaxScaler()
data_train['Total_Accounts'] = min_max_scalerT.fit_transform(  data_train['Total_Accounts'].fillna(-1).values.reshape(-1, 1) )

data_sub['Total_Accounts'] = min_max_scalerT.transform(  data_sub['Total_Accounts'].fillna(-1).values.reshape(-1, 1) )

# gender
data_train['Gender'] =  data_train['Gender'].str.replace('Male','1')
data_train['Gender'] =  data_train['Gender'].str.replace('Female','0')
data_train['Gender']  = data_train['Gender'] .astype(int)

data_sub['Gender'] =  data_sub['Gender'].str.replace('Male','1')
data_sub['Gender'] =  data_sub['Gender'].str.replace('Female','0')
data_sub['Gender']  = data_sub['Gender'] .astype(int)

di = {'not verified': 0, 'VERIFIED - income': 0.5,'VERIFIED - income source':1 }

data_train['Income_Verified'] = data_train['Income_Verified'].map(di)

data_sub['Income_Verified'] = data_sub['Income_Verified'].map(di)

dit = {'< 1 year': 0, '1 year': 0.1,'2 years': 0.2,'3 years': 0.3,'4 years': 0.4,'5 years': 0.5,'6 years': 0.6,'7 years': 0.7,'8 years': 0.8,'9 years': 0.9,'10+ years': 1}

data_train['Length_Employed'] = data_train['Length_Employed'].map(dit)

data_train['Length_Employed'] = data_train['Length_Employed'].fillna(-1)

data_sub['Length_Employed'] = data_sub['Length_Employed'].map(dit)

data_sub['Length_Employed'] = data_sub['Length_Employed'].fillna(-1)

dih = {'Own': 1, 'Mortgage': 0.75,'Rent': 0.5,'Other':0.25,'None':0}

data_train['Home_Owner'] = data_train['Home_Owner'].map(dih)
data_train['Home_Owner'] = data_train['Home_Owner'].fillna(-1)

data_sub['Home_Owner'] = data_sub['Home_Owner'].map(dih)
data_sub['Home_Owner'] = data_sub['Home_Owner'].fillna(-1)

lb_make = LabelEncoder()
data_train["Purpose_Of_Loan"] = lb_make.fit_transform(data_train["Purpose_Of_Loan"])

data_sub["Purpose_Of_Loan"] = lb_make.transform(data_sub["Purpose_Of_Loan"])

In [67]:
skf = StratifiedKFold(n_splits=5,random_state=42, shuffle=True)

In [68]:
X = data_train[['Loan_Amount_Requested','Annual_Income','Debt_To_Income','Inquiries_Last_6Mo','Months_Since_Deliquency','Number_Open_Accounts','Total_Accounts','Gender','Income_Verified','Length_Employed','Home_Owner']].values
y = data_train['Interest_Rate'].values
X2 = data_train[['Purpose_Of_Loan']].values


P = data_sub[['Loan_Amount_Requested','Annual_Income','Debt_To_Income','Inquiries_Last_6Mo','Months_Since_Deliquency','Number_Open_Accounts','Total_Accounts','Gender','Income_Verified','Length_Employed','Home_Owner']].values
P2 = data_sub[['Purpose_Of_Loan']].values

b = [] # append results
                
for train_index, test_index in skf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    
    X2_train,X2_test = X2[train_index], X2[test_index]
    
    # training lgbtm
    
    
    out = modelee.predict([X2_test,X_test])
    out2 = np.argmax(out,axis=1)
    print(f1_score(y_test, out2+1, average='weighted'))
    
    out = modelee.predict([P2,P])
    
    b.append(out)

NameError: name 'modelee' is not defined

In [108]:
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
import matplotlib.pyplot as plt

class EvoTreeOptimization(object):
    '''This class stores evolutionary optimization algorithms to trees based models'''
    
     # inicilaization function
    def __init__(self, cost_func, bounds, popsize=10, mutate=0.5, recombination=0.7, maxiter=2000):
        self.cost_func=cost_func
        self.bounds=bounds
        self.popsize=popsize
        self.mutate=mutate
        self.recombination=recombination
        self.maxiter=maxiter 
        self.bestInd = []
        self.meanpop = []
        self.best = []
        self.globvar = 0 # variable created to tune the output predicted value
        
        self.objective = 'xentropy'
        self.num_class = 1 
        self.f = self.lgbm_tra # function to be usded depending on multiclass/singleClass
           

    def ensure_bounds(self,vec, bounds):
        vec_new = []
        # cycle through each variable in vector
        for i in range(len(vec)):

            # variable exceedes the minimum boundary
            if vec[i] < bounds[i][0]:
                vec_new.append(bounds[i][0])

            # variable exceedes the maximum boundary
            if vec[i] > bounds[i][1]:
                vec_new.append(bounds[i][1])

            # the variable is fine
            if bounds[i][0] <= vec[i] <= bounds[i][1]:
                vec_new.append(vec[i])

        return vec_new
    
    def plotEvolution(self):
        plt.plot(trainX.meanpop)
        plt.plot(trainX.bestInd)
        plt.ylabel('cost function' + str(cost_func) )
        plt.ylabel('epochs')
        plt.show()

    
    def earlyFunc(self,preds, train_data):
        labels = train_data.get_label()
        return 'error',  self.cost_func(labels,np.round(preds-self.globvar)), True

    def lgbm_tra(self,x,X,y):
        
        # seleciona os parametros para treinamento
        params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': self.objective,
            'num_leaves': int(x[1]),      # 31
            'learning_rate': x[2],   #0.01,
            'feature_fraction': x[3],#0.9,
            'bagging_fraction': x[4],#0.8,
            'bagging_freq': 1,
            'max_depth': int(x[5]),        #-1,
            'min_data_in_leaf': int(x[6]), #20,
            'lambda_l2': x[7],        # 0,
            'is_unbalance' : True,
            'max_bin' : int(x[9]),
            'num_class' : self.num_class,
            'verbose': -1
        }
        
        num_k = 5
        acc = 0
        # treina a arvore
        skf = StratifiedKFold(n_splits=num_k,random_state=21)

        self.globvar = x[8]

        for train_index, test_index in skf.split(X,y):
           #print("TRAIN:", train_index, "TEST:", test_index)
           X_train, X_test = X[train_index,:], X[test_index,:]
           y_train, y_test = y[train_index], y[test_index]

           lgb_train = lgb.Dataset(X_train, y_train)
           
           print(X_train.shape)            
           print(y_train.shape)
        
           lgb_test = lgb.Dataset(X_test, y_test)
            
           print(X_test.shape)            
           print(y_test.shape)

           gbm = lgb.train(params, lgb_train, num_boost_round=int(x[0]),valid_sets = [lgb_test],early_stopping_rounds=30,feval = self.earlyFunc, verbose_eval= 0)     #x[0]

           y_pred = gbm.predict(X_test,num_iteration=gbm.best_iteration)
           acc += f1_score(y_test,np.round(y_pred-x[8]) ) # F1 score

        return (acc/num_k)

    def lgbm_tra_multi(self,x,X,y):
        
        # seleciona os parametros para treinamento
        params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': self.objective,
            'num_leaves': int(x[1]),      # 31
            'learning_rate': x[2],   #0.01,
            'feature_fraction': x[3],#0.9,
            'bagging_fraction': x[4],#0.8,
            'bagging_freq': 1,
            'max_depth': int(x[5]),        #-1,
            'min_data_in_leaf': int(x[6]), #20,
            'lambda_l2': x[7],        # 0,
            'is_unbalance' : True,
            'max_bin' : int(x[9]),
            'num_class' : self.num_class,
            'verbose': -1
        }
        
        num_k = 5
        acc = 0
        # treina a arvore
        skf = StratifiedKFold(n_splits=num_k)#,random_state=21)

        self.globvar = x[8]

        for train_index, test_index in skf.split(X,y):
           #print("TRAIN:", train_index, "TEST:", test_index)
           X_train, X_test = X[train_index,:], X[test_index,:]
           y_train, y_test = y[train_index], y[test_index]

           lgb_train = lgb.Dataset(X_train, y_train)           
        
           lgb_test = lgb.Dataset(X_test, y_test)            
        

           gbm = lgb.train(params, lgb_train, num_boost_round=int(x[0]),valid_sets = [lgb_test],early_stopping_rounds=30, verbose_eval= 0)     #x[0]

           y_pred = gbm.predict(X_test,num_iteration=gbm.best_iteration)
           acc += f1_score(y_test,np.argmax(y_pred,axis=1) ,average='weighted') # F1 score

        return (acc/num_k)


    def run(self,x_train,y_train):
        # --- INITIALIZE A POPULATION (step #1) ----------------+
        
        # check if is multiclass or single class
        
        if (max(y_train)) > 1:
            
            self.objective = 'multiclass'
            self.num_class = len(set(y_train)) 
            self.f = self.lgbm_tra_multi
        
        population = []
        for i in range(0, self.popsize):
            indv = []
            for j in range(len(self.bounds)):
                indv.append(random.uniform(self.bounds[j][0], self.bounds[j][1]))
            population.append(indv)

        print('First gen ...')
        scorePop = np.zeros(self.popsize)
        for num, ind in enumerate(population):
            scorePop[num] = self.f(ind, x_train, y_train)
            print(scorePop[num])

        # --- SOLVE --------------------------------------------+

        # cycle through each generation (step #2)
        for i in range(1, self.maxiter + 1):
            print('GENERATION:', i)

            gen_scores = []  # score keeping

            # cycle through each individual in the population
            for j in range(0, self.popsize):

                # --- MUTATION (step #3.A) ---------------------+

                # select three random vector index positions [0, popsize), not including current vector (j)
                candidates = np.arange(0, self.popsize)
                candidates = np.delete(candidates, j)

                random_index = np.random.permutation(candidates)

                x_1 = population[random_index[0]]
                x_2 = population[random_index[1]]
                x_3 = population[random_index[2]]
                x_t = population[j]  # target individual

                # subtract x3 from x2, and create a new vector (x_diff)
                x_diff = [x_2_i - x_3_i for x_2_i, x_3_i in zip(x_2, x_3)]

                # multiply x_diff by the mutation factor (F) and add to x_1
                v_donor = [x_1_i + self.mutate * x_diff_i for x_1_i, x_diff_i in zip(x_1, x_diff)]
                v_donor = self.ensure_bounds(v_donor, self.bounds)

                # --- RECOMBINATION (step #3.B) ----------------+

                v_trial = []
                for k in range(len(x_t)):
                    crossover = random.random()
                    if crossover <= self.recombination:
                        v_trial.append(v_donor[k])

                    else:
                        v_trial.append(x_t[k])

                # --- GREEDY SELECTION (step #3.C) -------------+

                score_trial = self.f(v_trial,x_train,y_train)
                score_target = scorePop[j]

                if score_trial > score_target:
                    population[j] = v_trial
                    scorePop[j] = score_trial
                    #print(score_target, '   <', score_trial, v_trial)

                else:
                    pass
                    #print(score_target,'   >', score_target, x_t)


            # --- SCORE KEEPING --------------------------------+
            gen_avg = np.mean(scorePop)  # current generation avg. fitness
            gen_best = max(scorePop)  # fitness of best individual
            gen_sol = population[np.argmax(scorePop)]  # solution of best individual

            print('      > GENERATION AVERAGE:', gen_avg)
            print('      > GENERATION BEST:', gen_best)
            print('         > BEST SOLUTION:', gen_sol, '\n')
            
            self.bestInd.append(gen_best)
            self.meanpop.append(gen_avg)
            self.best = gen_sol

        return gen_sol

In [109]:
cost_func = f1_score  # Cost function
#cost_func = accuracy_score
bounds = [(200, 1000),(15,50), (0.001,0.1 ), (0.5, 1),(0.7, 1), (6, 30),(10, 30),(0,20),(-0.4,0.4),(32,128)]  # Bounds [(x1_min, x1_max), (x2_min, x2_max),...]

In [110]:
trainX = EvoTreeOptimization(cost_func,bounds)

In [111]:
trainX.run(X_train,y_train-1)

First gen ...
0.5183525181290426
0.5096427284034795
0.48326378819882565
0.5182791643698614
0.5152553541376099
0.357825212213501
0.5174564846731868
0.5181440368982836
0.5177309869373563
0.5130813625216508
GENERATION: 1
      > GENERATION AVERAGE: 0.5170125512241222
      > GENERATION BEST: 0.5190501452672487
         > BEST SOLUTION: [549.1552690784881, 34.88364076246671, 0.057373210427928464, 0.7395869821604049, 0.9887678499600507, 15.86924892599761, 10, 6.875431112453276, 0.31725283764028034, 83.76960028479517] 

GENERATION: 2
      > GENERATION AVERAGE: 0.5178857450270831
      > GENERATION BEST: 0.5197804394880073
         > BEST SOLUTION: [537.2847219601083, 34.88364076246671, 0.07082028710527981, 0.7395869821604049, 0.8344131095396432, 15.86924892599761, 13.6341124874729, 6.875431112453276, 0.04451060507009391, 90.57013107877972] 

GENERATION: 3


KeyboardInterrupt: 

In [58]:
trainX.objective

'multiclass'

In [25]:
len(y_train)

131447

In [48]:
78870/26290

3.0

In [47]:
# seleciona os parametros para treinamento
        params = {
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': self.objective,
            'num_leaves': int(x[1]),      # 31
            'learning_rate': x[2],   #0.01,
            'feature_fraction': x[3],#0.9,
            'bagging_fraction': x[4],#0.8,
            'bagging_freq': 1,
            'max_depth': int(x[5]),        #-1,
            'min_data_in_leaf': int(x[6]), #20,
            'lambda_l2': x[7],        # 0,
            'is_unbalance' : True,
            'max_bin' : int(x[9]),
            'num_class' : self.num_class,
            'verbose': -1
        }

IndentationError: unexpected indent (<ipython-input-47-a897716a2c68>, line 2)

In [50]:
#importing libraries
import numpy as np
from collections import Counter
import pandas as pd
import lightgbm as lgb
from sklearn.datasets import load_breast_cancer,load_boston,load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score

In [52]:
#loading the dataset
X1=load_wine()
df_1=pd.DataFrame(X1.data,columns=X1.feature_names)
Y_1=X1.target#Scaling using the Standard Scaler
sc_1=StandardScaler()
sc_1.fit(df_1)
X_1=pd.DataFrame(sc_1.fit_transform(df_1))#train-test-split
X_train,X_test,y_train,y_test=train_test_split(X_1,Y_1,test_size=0.3,random_state=0)#Converting the dataset in proper LGB format
d_train=lgb.Dataset(X_train, label=y_train)#setting up the parameters
params={}
params['learning_rate']=0.03
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_class']=3 #no.of unique values in the target class not inclusive of the end value#training the model
clf=lgb.train(params,d_train,100)  #training the model on 100 epocs#prediction on the test dataset
y_pred_1=clf.predict(X_test)

In [56]:
X_train.shape

(124, 13)

In [55]:
y_train.shape

(124,)

In [81]:
# seleciona os parametros para treinamento
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'bagging_freq': 1,
    'is_unbalance' : True,
    'num_class' : 3,
    'verbose': -1
}


lgb_train = lgb.Dataset(X_train,y_train-1)
gbm = lgb.train(params, lgb_train, verbose_eval= 1)     #x[0]

In [83]:
gbm.predict(X_train)

array([[0.39914341, 0.47282652, 0.12803007],
       [0.09478797, 0.33043618, 0.57477585],
       [0.20177232, 0.39947447, 0.39875321],
       ...,
       [0.34032643, 0.53123315, 0.12844042],
       [0.17085772, 0.32768506, 0.50145723],
       [0.06011081, 0.37240119, 0.56748799]])